# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 8 2023 1:54PM,256965,10,MSP218372,"Methapharm, Inc.",Released
1,Feb 8 2023 1:54PM,256965,10,MSP218375,"Methapharm, Inc.",Released
2,Feb 8 2023 1:50PM,256964,10,Enova-11334,Emerginnova,Released
3,Feb 8 2023 1:49PM,256963,10,0086364366,ISDIN Corporation,Released
4,Feb 8 2023 1:49PM,256963,10,0086364363,ISDIN Corporation,Released
5,Feb 8 2023 1:49PM,256963,10,0086364365,ISDIN Corporation,Released
6,Feb 8 2023 1:49PM,256963,10,0086364362,ISDIN Corporation,Released
7,Feb 8 2023 1:49PM,256963,10,0086364947,ISDIN Corporation,Released
8,Feb 8 2023 1:49PM,256963,10,0086364948,ISDIN Corporation,Released
9,Feb 8 2023 1:49PM,256963,10,0086364957,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
51,256959,Released,1
52,256961,Released,5
53,256963,Released,18
54,256964,Released,1
55,256965,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256959,NaN,NaN,1.0
256961,NaN,NaN,5.0
256963,NaN,NaN,18.0
256964,NaN,NaN,1.0
256965,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,45.0,2.0,5.0
256826,0.0,3.0,110.0
256827,13.0,1.0,2.0
256857,0.0,1.0,10.0
256868,5.0,20.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,45,2,5
256826,0,3,110
256827,13,1,2
256857,0,1,10
256868,5,20,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,45,2,5
1,256826,0,3,110
2,256827,13,1,2
3,256857,0,1,10
4,256868,5,20,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,45,2,5
1,256826,,3,110
2,256827,13,1,2
3,256857,,1,10
4,256868,5,20,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 8 2023 1:54PM,256965,10,"Methapharm, Inc."
2,Feb 8 2023 1:50PM,256964,10,Emerginnova
3,Feb 8 2023 1:49PM,256963,10,ISDIN Corporation
21,Feb 8 2023 1:45PM,256961,19,"AdvaGen Pharma, Ltd"
26,Feb 8 2023 1:40PM,256959,19,VITABIOTICS USA INC.
27,Feb 8 2023 1:37PM,256958,19,VITABIOTICS USA INC.
28,Feb 8 2023 1:35PM,256957,10,"CLINUVEL, Inc."
29,Feb 8 2023 1:34PM,256956,12,Hush Hush
33,Feb 8 2023 1:33PM,256955,10,Yusen – 3D Matrix
38,Feb 8 2023 1:33PM,256954,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 8 2023 1:54PM,256965,10,"Methapharm, Inc.",,,2
1,Feb 8 2023 1:50PM,256964,10,Emerginnova,,,1
2,Feb 8 2023 1:49PM,256963,10,ISDIN Corporation,,,18
3,Feb 8 2023 1:45PM,256961,19,"AdvaGen Pharma, Ltd",,,5
4,Feb 8 2023 1:40PM,256959,19,VITABIOTICS USA INC.,,,1
5,Feb 8 2023 1:37PM,256958,19,VITABIOTICS USA INC.,,,1
6,Feb 8 2023 1:35PM,256957,10,"CLINUVEL, Inc.",,,1
7,Feb 8 2023 1:34PM,256956,12,Hush Hush,,,4
8,Feb 8 2023 1:33PM,256955,10,Yusen – 3D Matrix,,,5
9,Feb 8 2023 1:33PM,256954,12,Hush Hush,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 1:54PM,256965,10,"Methapharm, Inc.",2,,
1,Feb 8 2023 1:50PM,256964,10,Emerginnova,1,,
2,Feb 8 2023 1:49PM,256963,10,ISDIN Corporation,18,,
3,Feb 8 2023 1:45PM,256961,19,"AdvaGen Pharma, Ltd",5,,
4,Feb 8 2023 1:40PM,256959,19,VITABIOTICS USA INC.,1,,
5,Feb 8 2023 1:37PM,256958,19,VITABIOTICS USA INC.,1,,
6,Feb 8 2023 1:35PM,256957,10,"CLINUVEL, Inc.",1,,
7,Feb 8 2023 1:34PM,256956,12,Hush Hush,4,,
8,Feb 8 2023 1:33PM,256955,10,Yusen – 3D Matrix,5,,
9,Feb 8 2023 1:33PM,256954,12,Hush Hush,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 1:54PM,256965,10,"Methapharm, Inc.",2,,
1,Feb 8 2023 1:50PM,256964,10,Emerginnova,1,,
2,Feb 8 2023 1:49PM,256963,10,ISDIN Corporation,18,,
3,Feb 8 2023 1:45PM,256961,19,"AdvaGen Pharma, Ltd",5,,
4,Feb 8 2023 1:40PM,256959,19,VITABIOTICS USA INC.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 1:54PM,256965,10,"Methapharm, Inc.",2.0,NaN,NaN
1,Feb 8 2023 1:50PM,256964,10,Emerginnova,1.0,NaN,NaN
2,Feb 8 2023 1:49PM,256963,10,ISDIN Corporation,18.0,NaN,NaN
3,Feb 8 2023 1:45PM,256961,19,"AdvaGen Pharma, Ltd",5.0,NaN,NaN
4,Feb 8 2023 1:40PM,256959,19,VITABIOTICS USA INC.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 1:54PM,256965,10,"Methapharm, Inc.",2.0,0.0,0.0
1,Feb 8 2023 1:50PM,256964,10,Emerginnova,1.0,0.0,0.0
2,Feb 8 2023 1:49PM,256963,10,ISDIN Corporation,18.0,0.0,0.0
3,Feb 8 2023 1:45PM,256961,19,"AdvaGen Pharma, Ltd",5.0,0.0,0.0
4,Feb 8 2023 1:40PM,256959,19,VITABIOTICS USA INC.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5909396,121.0,2.0,0.0
12,513910,5.0,0.0,0.0
15,1284271,121.0,21.0,74.0
16,256873,0.0,1.0,0.0
19,2569184,21.0,22.0,5.0
20,256930,1.0,0.0,0.0
21,256870,1.0,0.0,0.0
22,256918,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5909396,121.0,2.0,0.0
1,12,513910,5.0,0.0,0.0
2,15,1284271,121.0,21.0,74.0
3,16,256873,0.0,1.0,0.0
4,19,2569184,21.0,22.0,5.0
5,20,256930,1.0,0.0,0.0
6,21,256870,1.0,0.0,0.0
7,22,256918,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,121.0,2.0,0.0
1,12,5.0,0.0,0.0
2,15,121.0,21.0,74.0
3,16,0.0,1.0,0.0
4,19,21.0,22.0,5.0
5,20,1.0,0.0,0.0
6,21,1.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,121.0
1,12,Released,5.0
2,15,Released,121.0
3,16,Released,0.0
4,19,Released,21.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,74.0,0.0,5.0,0.0,0.0,0.0
Executing,2.0,0.0,21.0,1.0,22.0,0.0,0.0,0.0
Released,121.0,5.0,121.0,0.0,21.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,74.0,0.0,5.0,0.0,0.0,0.0
1,Executing,2.0,0.0,21.0,1.0,22.0,0.0,0.0,0.0
2,Released,121.0,5.0,121.0,0.0,21.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,74.0,0.0,5.0,0.0,0.0,0.0
1,Executing,2.0,0.0,21.0,1.0,22.0,0.0,0.0,0.0
2,Released,121.0,5.0,121.0,0.0,21.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()